# DICOM Standard harvesting of Part 3 
## Extract relationships between tags and CIDs

[Useful Online XML Viewer](https://jsonformatter.org/xml-viewer)

Links to the XML Objects
- [DICOM Part 3](https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml)
- [DICOM Part 6](https://dicom.nema.org/medical/dicom/current/source/docbook/part06/part06.xml)
- [DICOM Part 16](https://dicom.nema.org/medical/dicom/current/source/docbook/part16/part16.xml)

This notebook demonstrates how to parse out DICOM Part 3. 
1. Extract one IOD module table: A.36 Enhanced MR and its Reference tables
2. Extract all IOD module tables and their Reference tables


## 1.a Extract one IOD Module Table: A.36 Enhanced MR

In [ ]:
from bs4 import BeautifulSoup

# Fetch XML content from the URL
url = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml'
response = requests.get(url)
xml_content = response.content

# Parse the XML content
soup = BeautifulSoup(xml_content, 'xml')

In [1]:
import requests
import pandas as pd

# Find the table with label 'A.36-1'
table = soup.find('table', {'label': 'A.36-1'})

# Find the section containing the table and extract its title
section_title = soup.find('section', {'xml:id': table.parent['xml:id']}).find('title').text.strip()

# Extract the part of the title before "IOD"
iod_index = section_title.find(" IOD")
iod = section_title[:iod_index]

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
rows = []
current_ie = None
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all(['td', 'th'])

    # Extract cell values
    cell_values = [cell.text.strip() for cell in cells]

    # Check if the cell values are complete (some cells may be missing due to rowspan)
    if len(cell_values) < len(headers):
        # Fill in the missing values based on the previous row
        for i in range(len(headers) - len(cell_values)):
            cell_values.insert(0, current_ie)

    # Store the IE value for the next iteration
    current_ie = cell_values[0]

    # Extract the reference value if available
    reference = None
    for cell in cells:
        xref = cell.find('xref', {'xrefstyle': 'select: labelnumber'})
        if xref:
            reference = xref['linkend']
            break

    # Insert the reference value into the correct position
    if reference:
        # Find the index of the cell containing the reference
        reference_index = next((idx for idx, val in enumerate(cell_values) if val == ''), None)
        if reference_index is not None:
            cell_values[reference_index] = reference

    # Insert the title of the table as the first element
    cell_values.insert(0, iod)

    rows.append(cell_values)

# Create a DataFrame
df = pd.DataFrame(rows, columns=['IOD'] + headers)


/Users/wooyeonpark/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df

,IOD,IE,Module,Reference,Usage
0,Enhanced MR Image,Patient,Patient,sect_C.7.1.1,M
1,Enhanced MR Image,Patient,Clinical Trial Subject,sect_C.7.1.3,U
2,Enhanced MR Image,Study,General Study,sect_C.7.2.1,M
3,Enhanced MR Image,Study,Patient Study,sect_C.7.2.2,U
4,Enhanced MR Image,Study,Clinical Trial Study,sect_C.7.2.3,U
5,Enhanced MR Image,Series,General Series,sect_C.7.3.1,M
6,Enhanced MR Image,Series,Clinical Trial Series,sect_C.7.3.2,U
7,Enhanced MR Image,Series,MR Series,sect_C.8.13.6,M
8,Enhanced MR Image,Frame of Reference,Frame of Reference,sect_C.7.4.1,M
9,Enhanced MR Image,Frame of Reference,Synchronization,sect_C.7.4.2,C - Required if time synchronization was applied.


## 1.b Extract Reference tables

In [ ]:
# Find the mapping table between reference module from IOD table to reference table
# this one is from A.36 Enhanced MR IOD 
rows = []

df_reference = df[df['Usage']=="M"]

for section_id in df_reference['Reference']:
    section_element = soup.find('section', {'xml:id': section_id})
    
    if section_element:
        tables = section_element.findChildren('table', recursive=False)
        # For each table, create a dictionary with section_id and table XML id and add it to the rows list
        for table in tables:
            if table.has_attr('xml:id'):
                row = {'section_id': section_id, 'table_xml_id': table['xml:id']}
                rows.append(row)
# Create a DataFrame from the rows list
table_df = pd.DataFrame(rows)
table_df

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

def process_table(soup, table_xml_id):
    # Find the table using table_xml_id
    table = soup.find('table', {'xml:id': table_xml_id})
    table_xml_id = table.parent['xml:id']

    # Initialize lists to store extracted data
    attribute_names = []
    tags = []
    types = []
    attribute_descriptions = []
    cid = []

    # Extract data from the table rows
    for index, row in enumerate(table.find_all('tr')):
        columns = row.find_all('td')
        if len(columns) == 4:
            attribute_name = columns[0].text.strip()
            if attribute_name.startswith('>'):
                continue
            tag = columns[1].text.strip()
            type_ = columns[2].text.strip()
            description = columns[3].find_all('variablelist')

            defined_terms_dict = {}
            if description:
                for variablelist in description:
                    title = variablelist.title.text.strip().replace(':', '')
                    defined_terms = [term.text.strip() for term in variablelist.find_all('term')]
                    defined_terms_dict[title] = defined_terms
            
            attribute_names.append(attribute_name)
            tags.append(tag)
            types.append(type_)
            attribute_descriptions.append(defined_terms_dict)
            cid.append('')

        elif len(columns) == 2 and index != 0 and cid:
            first_col = columns[0].text.strip()
            if first_col.startswith('>>'):
                continue
            if first_col.startswith('>'):
                olink = columns[1].find('olink')
                if olink:
                    targetptr = olink.get('targetptr', '')
                    if 'CID_' in targetptr:
                        cid_value = targetptr.split('CID_')[-1]
                        cid[-1] += cid_value

    # Create a DataFrame from the extracted data
    data = {
        'section_id': table_xml_id,
        'Attribute Name': attribute_names,
        'Tag': tags,
        'Type': types,
        'Attribute Description': attribute_descriptions,
        'CID': cid
    }
    return pd.DataFrame(data)

In [ ]:
reference_table_data = []
for table_xml_id in table_df['table_xml_id']:
    df = process_table(soup, table_xml_id)
    reference_table_data.append(df)

# combine all DataFrames into one
reference_tables = pd.concat(reference_table_data, ignore_index=True)

## 2.a Extract Module Tables for all IODs

In [48]:
from bs4 import BeautifulSoup

# Fetch XML content from the URL
url = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml'
response = requests.get(url)
xml_content = response.content

# Parse the XML content
soup = BeautifulSoup(xml_content, 'xml')

In [83]:
import re
from bs4 import BeautifulSoup
import pandas as pd

data = []

for section_element in soup.find_all('section', {'label': re.compile('A.')}):
    # Find all table elements within the section_element
    tables = section_element.find_all('table')
    for table in tables:
        # Find the caption element for the current table
        caption = table.find('caption')
        # Check if the caption exists and matches the regular expression
        if caption and re.search('IOD Modules', caption.text):
            # Extract the xml:id attribute from the table
            table_id = table.get('xml:id')  # Use .get() to avoid KeyError if the attribute is missing
            # Append the xml:id and caption text to the data list
            data.append((table_id, caption.text.strip()))

# Create a DataFrame from the collected data
df_iod_tables = pd.DataFrame(data, columns=['xml_id', 'iod'])
df_iod_tables = df_iod_tables.drop_duplicates().reset_index(drop=True)


In [84]:
df_iod_tables

,xml_id,iod
0,table_A.2-1,Computed Radiography Image IOD Modules
1,table_A.3-1,CT Image IOD Modules
2,table_A.4-1,MR Image IOD Modules
3,table_A.5-1,Nuclear Medicine Image IOD Modules
4,table_A.6-1,Ultrasound Image IOD Modules
...,...,...
165,table_A.87-1,Microscopy Bulk Simple Annotations IOD Modules
166,table_A.88.3-1,Inventory IOD Modules
167,table_A.89.3-1,Photoacoustic Image IOD Modules
168,table_A.90.1.3-1,Confocal Microscopy Image IOD Modules


In [53]:
import pandas as pd

first_table_headers = None
table_data = []  # A list to store the DataFrame for each table

for index, row in df_iod_tables.iterrows():
    xml_id = row['xml_id']
    caption = row['iod']

    # Find the table in 'soup' with the matching 'xml:id'
    table = soup.find('table', {'xml:id': xml_id})
    if table:
        # Extract table headers
        headers = [th.text.strip() for th in table.find_all('th')]
        # If it's the first table, store its headers
        if first_table_headers is None:
            first_table_headers = headers
        # For subsequent tables, check if the headers match the first table's headers
        elif headers != first_table_headers:
            # If headers don't match, skip this table
            continue

        # Extract table rows
        rows = []
        for tr in table.find_all('tr')[1:]:  # Skip the header row
            cells = tr.find_all(['td', 'th'])
            # Extract cell values
            cell_values = [cell.text.strip() for cell in cells]

            # Check if the cell values are complete (some cells may be missing due to rowspan)
            if len(cell_values) < len(headers):
                # Fill in the missing values based on the previous row
                for i in range(len(headers) - len(cell_values)):
                    cell_values.insert(0, current_ie)

            # Extract the reference value if available
            reference = None
            for cell in cells:
                xref = cell.find('xref', {'xrefstyle': 'select: labelnumber'})
                if xref:
                    reference = xref['linkend']
                    break

            # Insert the reference value into the correct position
            if reference:
                # Find the index of the cell containing the reference
                reference_index = next((idx for idx, val in enumerate(cell_values) if val == ''), None)
                if reference_index is not None:
                    cell_values[reference_index] = reference
            table_data.append([xml_id, caption] + cell_values)

final_df = pd.DataFrame(table_data, columns=['xml_id', 'iod'] + first_table_headers)


In [85]:
final_df['Usage_code'] = final_df['Usage'].str[0]
final_df

,xml_id,iod,IE,Module,Reference,Usage,Usage_code
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M
1,table_A.2-1,Computed Radiography Image IOD Modules,Image,Clinical Trial Subject,sect_C.7.1.3,U,U
2,table_A.2-1,Computed Radiography Image IOD Modules,Study,General Study,sect_C.7.2.1,M,M
3,table_A.2-1,Computed Radiography Image IOD Modules,Image,Patient Study,sect_C.7.2.2,U,U
4,table_A.2-1,Computed Radiography Image IOD Modules,Image,Clinical Trial Study,sect_C.7.2.3,U,U
...,...,...,...,...,...,...,...
3248,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Cutaneous Confocal Microscopy Image Acquisitio...,sect_C.8.35.3,C - Required for cutaneous confocal microscopy,C
3249,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Optical Path,sect_C.8.12.5,M,M
3250,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M
3251,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U


In [92]:
final_df.groupby('Usage_code')['xml_id'].count()

Usage_code
C     314
M    1760
U    1179
Name: xml_id, dtype: int64

In [57]:
final_df['xml_id'].nunique()

170

In [58]:
final_df['Reference'].nunique()

358

### Exception handling
It looks like we have a list of Reference tables that needed further adjustments. Upon investigation, I found that the standard for Reference tables above redirect to another table or tables. To handle this exception, I have manually collected the redirected References and created a dataframe as mapping_data.

In [119]:
#'sect_C.8.15.4': 'sect_C.8.2.2.1', 'sect_C.8.2.2.2', 'sect_C.8.2.2.3'
#'sect_C.11.11': 'sect_C.11.11.1'
#'sect_C.17.3': 'sect_C.17.3.3', 'sect_C.17.3.4'
#'sect_C.7.6.20': 'sect_C.10.12'
#'sect_C.28.1':'sect_C.10.9'
#'sect_C.29.3':'sect_C.29.3.1'
#'sect_C.36.18': 'sect_C.36.2.2.7', 'sect_C.36.2.2.8', 'sect_C.36.2.2.14'

mapping_data = {
    'Reference': ['sect_C.8.15.4', 'sect_C.8.15.4', 'sect_C.8.15.4', 'sect_C.11.11', 'sect_C.17.3', 'sect_C.17.3', 'sect_C.7.6.20', 'sect_C.28.1', 'sect_C.29.3', 'sect_C.36.18', 'sect_C.36.18', 'sect_C.36.18'],
    'Reference_adjusted': ['sect_C.8.2.2.1', 'sect_C.8.2.2.2', 'sect_C.8.2.2.3', 'sect_C.11.11.1', 'sect_C.17.3.3', 'sect_C.17.3.4', 'sect_C.10.12', 'sect_C.10.9', 'sect_C.29.3.1', 'sect_C.36.2.2.7', 'sect_C.36.2.2.8', 'sect_C.36.2.2.14']
}
mapping_data = pd.DataFrame(mapping_data)
mapping_data

,Reference,Reference_adjusted
0,sect_C.8.15.4,sect_C.8.2.2.1
1,sect_C.8.15.4,sect_C.8.2.2.2
2,sect_C.8.15.4,sect_C.8.2.2.3
3,sect_C.11.11,sect_C.11.11.1
4,sect_C.17.3,sect_C.17.3.3
5,sect_C.17.3,sect_C.17.3.4
6,sect_C.7.6.20,sect_C.10.12
7,sect_C.28.1,sect_C.10.9
8,sect_C.29.3,sect_C.29.3.1
9,sect_C.36.18,sect_C.36.2.2.7


In [122]:
final_df_expanded = pd.merge(final_df, mapping_data, on = 'Reference', how = 'left')
final_df_expanded['Reference_adjusted'] = final_df_expanded['Reference_adjusted'].fillna(final_df_expanded['Reference'])
final_df_expanded

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1
1,table_A.2-1,Computed Radiography Image IOD Modules,Image,Clinical Trial Subject,sect_C.7.1.3,U,U,sect_C.7.1.3
2,table_A.2-1,Computed Radiography Image IOD Modules,Study,General Study,sect_C.7.2.1,M,M,sect_C.7.2.1
3,table_A.2-1,Computed Radiography Image IOD Modules,Image,Patient Study,sect_C.7.2.2,U,U,sect_C.7.2.2
4,table_A.2-1,Computed Radiography Image IOD Modules,Image,Clinical Trial Study,sect_C.7.2.3,U,U,sect_C.7.2.3
...,...,...,...,...,...,...,...,...
3276,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Cutaneous Confocal Microscopy Image Acquisitio...,sect_C.8.35.3,C - Required for cutaneous confocal microscopy,C,sect_C.8.35.3
3277,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Optical Path,sect_C.8.12.5,M,M,sect_C.8.12.5
3278,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M,sect_C.12.1
3279,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U,sect_C.12.2


## 2.b Extract all Reference tables

In [124]:
# Find the mapping table between reference module from IOD table to reference table
# Find from the all IOD list for ALL usage
rows = []

df_reference = final_df_expanded['Reference_adjusted'].drop_duplicates()

for section_id in df_reference:
    section_element = soup.find('section', {'xml:id': section_id})
    
    if section_element:
        tables = section_element.findChildren('table', recursive=False)
        # For each table, create a dictionary with section_id and table XML id and add it to the rows list
        for table in tables:
            if table.has_attr('xml:id'):
                row = {'section_id': section_id, 'table_xml_id': table['xml:id']}
                rows.append(row)
# Create a DataFrame from the rows list
table_df = pd.DataFrame(rows)
table_df

,section_id,table_xml_id
0,sect_C.7.1.1,table_C.7-1
1,sect_C.7.1.3,table_C.7-2b
2,sect_C.7.2.1,table_C.7-3
3,sect_C.7.2.2,table_C.7-4a
4,sect_C.7.2.3,table_C.7-4b
...,...,...
371,sect_C.8.34.3,table_C.8.34.3-1
372,sect_C.8.34.4,table_C.8.34.4-1
373,sect_C.8.35.1,table_C.8.35.1-1
374,sect_C.8.35.3,table_C.8.35.3-1


In [75]:
# same function as above 
import pandas as pd
from bs4 import BeautifulSoup

def process_table(soup, table_xml_id):
    # Find the table using table_xml_id
    table = soup.find('table', {'xml:id': table_xml_id})
    table_xml_id = table.parent['xml:id']

    # Initialize lists to store extracted data
    attribute_names = []
    tags = []
    types = []
    attribute_descriptions = []
    cid = []

    # Extract data from the table rows
    for index, row in enumerate(table.find_all('tr')):
        columns = row.find_all('td')
        if len(columns) == 4:
            attribute_name = columns[0].text.strip()
            if attribute_name.startswith('>'):
                continue
            tag = columns[1].text.strip()
            type_ = columns[2].text.strip()
            description = columns[3].find_all('variablelist')

            defined_terms_dict = {}
            if description:
                for variablelist in description:
                    title = variablelist.title.text.strip().replace(':', '')
                    defined_terms = [term.text.strip() for term in variablelist.find_all('term')]
                    defined_terms_dict[title] = defined_terms
            
            attribute_names.append(attribute_name)
            tags.append(tag)
            types.append(type_)
            attribute_descriptions.append(defined_terms_dict)
            cid.append('')

        elif len(columns) == 2 and index != 0 and cid:
            first_col = columns[0].text.strip()
            if first_col.startswith('>>'):
                continue
            if first_col.startswith('>'):
                olink = columns[1].find('olink')
                if olink:
                    targetptr = olink.get('targetptr', '')
                    if 'CID_' in targetptr:
                        cid_value = targetptr.split('CID_')[-1]
                        cid[-1] += cid_value

    # Create a DataFrame from the extracted data
    data = {
        'section_id': table_xml_id,
        'Attribute Name': attribute_names,
        'Tag': tags,
        'Type': types,
        'Attribute Description': attribute_descriptions,
        'CID': cid
    }
    return pd.DataFrame(data)

In [125]:
reference_table_data = []
for table_xml_id in table_df['table_xml_id']:
    df = process_table(soup, table_xml_id)
    reference_table_data.append(df)

# combine all DataFrames into one
reference_tables = pd.concat(reference_table_data, ignore_index=True)

In [126]:
reference_tables

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
0,sect_C.7.1.1,Patient's Name,"(0010,0010)",2,{},
1,sect_C.7.1.1,Patient ID,"(0010,0020)",2,{},
2,sect_C.7.1.1,Type of Patient ID,"(0010,0022)",3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
3,sect_C.7.1.1,Patient's Birth Date,"(0010,0030)",2,{},
4,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,"(0010,0033)",3,{},
...,...,...,...,...,...,...
2688,sect_C.8.35.3,Tracking UID,"(0062,0021)",1C,{},
2689,sect_C.8.35.2,Imaged Volume Width,"(0048,0001)",1,{},
2690,sect_C.8.35.2,Imaged Volume Height,"(0048,0002)",1,{},
2691,sect_C.8.35.2,Imaged Volume Depth,"(0048,0003)",1,{},


In [127]:
# subset of tags based on type = required
reference_tables[reference_tables['Type']=='1']

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
33,sect_C.7.1.3,Clinical Trial Sponsor Name,"(0012,0010)",1,{},
34,sect_C.7.1.3,Clinical Trial Protocol ID,"(0012,0020)",1,{},
42,sect_C.7.2.1,Study Instance UID,"(0020,000D)",1,{},
93,sect_C.7.3.1,Modality,"(0008,0060)",1,{},
94,sect_C.7.3.1,Series Instance UID,"(0020,000E)",1,{},
...,...,...,...,...,...,...
2682,sect_C.8.35.1,Tissue Location,"(0048,0115)",1,"{'Enumerated Values': ['INVIVO', 'EXVIVO']}",
2689,sect_C.8.35.2,Imaged Volume Width,"(0048,0001)",1,{},
2690,sect_C.8.35.2,Imaged Volume Height,"(0048,0002)",1,{},
2691,sect_C.8.35.2,Imaged Volume Depth,"(0048,0003)",1,{},


In [128]:
# subset of tags with attribute description
reference_tables[reference_tables['Attribute Description']!= {}]

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
2,sect_C.7.1.1,Type of Patient ID,"(0010,0022)",3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
7,sect_C.7.1.1,Patient's Sex,"(0010,0040)",2,"{'Enumerated Values': ['M', 'F', 'O']}",
9,sect_C.7.1.1,Quality Control Subject,"(0010,0200)",3,"{'Enumerated Values': ['YES', 'NO']}",
30,sect_C.7.1.1,Patient Identity Removed,"(0012,0062)",3,"{'Enumerated Values': ['YES', 'NO']}",
73,sect_C.7.2.2,Smoking Status,"(0010,21A0)",3,"{'Enumerated Values': ['YES', 'NO', 'UNKNOWN']}",
...,...,...,...,...,...,...
2680,sect_C.8.35.1,Lossy Image Compression,"(0028,2110)",1,"{'Enumerated Values': ['00', '01']}",
2681,sect_C.8.35.1,Confocal Mode,"(0048,0114)",1,"{'Enumerated Values': ['REFLECTANCE', 'FLUORES...",
2682,sect_C.8.35.1,Tissue Location,"(0048,0115)",1,"{'Enumerated Values': ['INVIVO', 'EXVIVO']}",
2685,sect_C.8.35.3,Field of View Shape,"(0018,1147)",2,{'Defined Terms': ['RECTANGLE']},


In [129]:
# subset of tags with CIDs
reference_tables[reference_tables['CID']!='']

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
17,sect_C.7.1.1,Patient Species Code Sequence,"(0010,2202)",1C,{},7454
19,sect_C.7.1.1,Patient Breed Code Sequence,"(0010,2293)",2C,{},7480
32,sect_C.7.1.1,De-identification Method Code Sequence,"(0012,0064)",1C,{},7050
58,sect_C.7.2.1,Requesting Service Code Sequence,"(0032,1034)",3,{},7030
60,sect_C.7.2.1,Procedure Code Sequence,"(0008,1032)",3,{},101
...,...,...,...,...,...,...
2664,sect_C.8.34.2,Illumination Type Code Sequence,"(0022,0016)",3,{},11001
2666,sect_C.8.34.2,Acoustic Coupling Medium Code Sequence,"(0018,982A)",2C,{},11002
2668,sect_C.8.34.3,Transducer Geometry Code Sequence,"(0018,980D)",1,{},12033
2670,sect_C.8.34.3,Transducer Technology Sequence,"(0018,9831)",3,{},11003


In [132]:
# create one dataframe and export the file
mapping_table = pd.merge(final_df_expanded, reference_tables, left_on = 'Reference_adjusted', right_on = 'section_id', how = 'left')
mapping_table

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted,section_id,Attribute Name,Tag,Type,Attribute Description,CID
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient's Name,"(0010,0010)",2,{},
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient ID,"(0010,0020)",2,{},
2,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Type of Patient ID,"(0010,0022)",3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
3,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient's Birth Date,"(0010,0030)",2,{},
4,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,"(0010,0033)",3,{},
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37052,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M,sect_C.12.1,sect_C.12.1,Instance Origin Status,"(0400,0600)",3,"{'Enumerated Values': ['LOCAL', 'IMPORTED']}",
37053,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M,sect_C.12.1,sect_C.12.1,Barcode Value,"(2200,0005)",3,{},
37054,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U,sect_C.12.2,sect_C.12.2,Referenced Series Sequence,"(0008,1115)",1C,{},
37055,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U,sect_C.12.2,sect_C.12.2,Studies Containing Other Referenced Instances ...,"(0008,1200)",1C,{},


In [ ]:
mapping_table.to_csv('./files/part3_mapping.csv', index=False)

In [ ]:
mapping_table.to_pickle('./files/part3_mapping.pkl')